# Chord Detection Notebook
This notebook demonstrates an end-to-end pipeline for detecting musical chords from audio files. It covers audio preprocessing, feature extraction, chord recognition, post-processing, batch file handling, and exporting data for Swift integration.

In [1]:
import os
import json
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from collections import Counter


In [2]:
PITCH_CLASSES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def generate_chord_templates():
    templates = {}
    for i, root in enumerate(PITCH_CLASSES):
        major = np.zeros(12)
        minor = np.zeros(12)
        major[[i, (i+4)%12, (i+7)%12]] = 1
        minor[[i, (i+3)%12, (i+7)%12]] = 1
        templates[f"{root}:maj"] = major
        templates[f"{root}:min"] = minor
        
    return templates

CHORD_TEMPLATES = generate_chord_templates()


In [3]:
def preprocess_audio(path, target_sr=22050):
    y, sr = librosa.load(path, sr=target_sr, mono=True)
    y = librosa.util.normalize(y)
    y, _ = librosa.effects.trim(y)
    return y, sr


In [4]:
def extract_features(y, sr):
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    pitches = librosa.yin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    return chroma, pitches


In [5]:
def recognize_chords(chroma, sr, hop_length=512):
    time_stamps = librosa.frames_to_time(np.arange(chroma.shape[1]), sr=sr, hop_length=hop_length)
    chords = []
    confidences = []
    for frame in chroma.T:
        scores = {chord: np.dot(frame, template) for chord, template in CHORD_TEMPLATES.items()}
        chord = max(scores, key=scores.get)
        chords.append(chord)
        confidences.append(scores[chord])
    return time_stamps, chords, confidences


In [6]:
def smooth_chords(chords, confidences, time_stamps):
    smoothed = []
    times = []
    confs = []
    prev = None
    for chord, conf, t in zip(chords, confidences, time_stamps):
        if chord != prev:
            smoothed.append(chord)
            times.append(t)
            confs.append(conf)
            prev = chord
        else:
            confs[-1] = max(confs[-1], conf)
    return times, smoothed, confs


In [7]:
def process_file(path):
    y, sr = preprocess_audio(path)
    chroma, pitches = extract_features(y, sr)
    times, chords, confs = recognize_chords(chroma, sr)
    times, chords, confs = smooth_chords(chords, confs, times)
    key = PITCH_CLASSES[np.argmax(chroma.mean(axis=1))]
    progression = '-'.join(chords)
    mean_conf = float(np.mean(confs))
    df_prog = pd.DataFrame({'time': times, 'chord': chords, 'confidence': confs})
    return {'song': os.path.basename(path), 'nada_dasar': key, 'chord_progression': progression, 'confidence': mean_conf}, df_prog


def process_batch(paths, summary_csv='songs_summary.csv', progression_dir='progressions'):
    os.makedirs(progression_dir, exist_ok=True)
    summaries = []
    for path in paths:
        summary, df = process_file(path)
        summaries.append(summary)
        stem = os.path.splitext(os.path.basename(path))[0]
        df.to_csv(os.path.join(progression_dir, f'{stem}_chords.csv'), index=False)
    pd.DataFrame(summaries).to_csv(summary_csv, index=False)
    return summaries


In [8]:
def export_templates_json(json_path='chord_templates.json'):
    with open(json_path, 'w') as f:
        json.dump({k: v.tolist() for k, v in CHORD_TEMPLATES.items()}, f, indent=2)
    return json_path


In [9]:
# Example usage with a synthetic audio file
sr = 22050

# Generate a C major chord followed by a G major chord
def chord_tone(notes, sr, duration):
    t = np.linspace(0, duration, int(sr*duration), False)
    audio = sum(librosa.tone(librosa.note_to_hz(n), sr=sr, length=len(t)) for n in notes)
    return audio / np.max(np.abs(audio))

audio = np.concatenate([
    chord_tone(['C3','E3','G3'], sr, 1.0),
    chord_tone(['G3','B3','D4'], sr, 1.0)
])

sf.write('demo.wav', audio, sr)
process_batch(['demo.wav'])


/root/.pyenv/versions/3.11.12/lib/python3.11/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=690
  warnings.warn(


[{'song': 'demo.wav',
  'nada_dasar': 'G',
  'chord_progression': 'C:maj-G:maj',
  'confidence': 2.830510228872299}]

In [10]:
import shutil, os
os.remove('demo.wav')
os.remove('songs_summary.csv')
shutil.rmtree('progressions')
if os.path.exists('chord_templates.json'):
    os.remove('chord_templates.json')
